## Passo a passo
1. Criar um wrapper para cada site
  - Criação do conjunto rotulado
2. Implementar uma solução para todos os sites
  - Extra: implementar mais uma solução e fazer análise de erros
3. Comparar estratégias (Accuracy, precision, recall...)

## Dependências

In [1]:
import re
from urllib.request import urlopen, build_opener
from bs4 import BeautifulSoup

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
base_path = '/content/drive/MyDrive/2022.1/RI'

## Extratores

### Amazon

#### Testes

In [ ]:
file = f'{base_path}/amazon/2022-08-07_18.16.30.962588.html'
html = open(file,'r')
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
title = soup.find('span', id='productTitle').string.strip('\n ')

In [ ]:
title

'Disciplina É Liberdade: Manual do Campo'

In [ ]:
description_div = soup.find('div', id='bookDescription_feature_div')
description_div

<div class="celwidget" data-csa-c-id="u3yptp-u0uv3c-xgklqn-3nbu45" data-feature-name="bookDescription" id="bookDescription_feature_div">
<div class="a-expander-collapsed-height a-row a-expander-container a-spacing-base a-expander-partial-collapse-container" data-a-expander-collapsed-height="140" data-a-expander-name="book_description_expander" style="max-height: none; height: 140px;"> <div aria-expanded="false" class="a-expander-content a-expander-partial-collapse-content" style="padding-bottom: 20px;"> <span>Os métodos de Jocko Willink para o sucesso nasceram nas equipes SEAL da marinha norte-americana, onde passou grande parte de sua vida adulta, alistando-se depois de sair do ensino médio e subindo na hierarquia até se tornar comandante das operações especiais com as mais altas condecorações na Guerra do Iraque. Em Disciplina É Liberdade, o coautor do livro Responsabilidade Extrema, best-seller do New York Times, descreve como vive segundo este mantra: as disciplinas mental e física

In [ ]:
info = description_div.div.div.span.string.strip('\n ')
info

'Os métodos de Jocko Willink para o sucesso nasceram nas equipes SEAL da marinha norte-americana, onde passou grande parte de sua vida adulta, alistando-se depois de sair do ensino médio e subindo na hierarquia até se tornar comandante das operações especiais com as mais altas condecorações na Guerra do Iraque. Em Disciplina É Liberdade, o coautor do livro Responsabilidade Extrema, best-seller do New York Times, descreve como vive segundo este mantra: as disciplinas mental e física que ele impõe a si mesmo para ter liberdade em todos os aspectos da vida. Muitos livros aconselham como superar obstáculos e alcançar seus objetivos, mas tal conselho costuma omitir o ingrediente mais crítico: a disciplina. Sem ela, não há progresso real. A obra Disciplina É Liberdade cobre tudo, inclusive estratégias e táticas para vencer a fraqueza, a procrastinação e o medo. Também apresenta um treinamento físico específico com exercícios para atletas iniciantes, intermediários e avançados, assim como mel

In [ ]:
authors_div = soup.find('div', id='bylineInfo')
authors_div
authors_spans = authors_div.find_all('span', class_='author')
authors_spans

[<span class="author notFaded" data-width="">
 <div class="a-popover-preload" id="a-popover-contributor-info-B00ZY7MWW8"> <div class="a-section"> <table class="a-normal"> <tbody><tr> <td id="contributorImageContainerB00ZY7MWW8"></td> <script data-a-state='{"key":"popoverImage-state"}' type="a-state">{"isAjaxComplete_B00ZY7MWW8":"0","isAjaxInProgress_B00ZY7MWW8":"0"}</script> <td class="a-size-base"> <div class="a-row"> <span class="a-size-medium">Jocko Willink  <span class="a-color-secondary">(Autor)</span> </span> </div> <div class="a-row"> <span class="a-text-bold">› </span><a class="a-link-normal" href="/Jocko-Willink/e/B00ZY7MWW8/ref=dp_byline_cont_pop_book_1">Visite a página de Jocko Willink</a> </div> <div class="a-row a-spacing-base"> Encontre todos os livros, leia sobre o autor, e muito mais. </div> <div class="a-row"> <span class="a-size-small">Consulte <a class="a-link-normal" href="/s/ref=dp_byline_sr_pop_book_1?ie=UTF8&amp;field-author=Jocko+Willink&amp;text=Jocko+Willink&a

In [ ]:
authors = []
for author in authors_spans:
  if 'Autor' in str(author.find('span', class_='contribution').span.string):
    if author.a:
      result = str(author.a.string)
      authors.append(result if 'Visite' not in result else result.replace('Visite a página de ', ''))

In [ ]:
authors

['Jocko Willink']

In [ ]:
price_span = soup.find('span', id='price')
price_span

<span class="a-size-medium a-color-price header-price a-text-normal" id="price">R$ 47,93</span>

In [ ]:
price = ''
if price_span:
  price = str(price_span.string).replace(u'\xa0', u' ')

price

'R$ 47,93'

In [ ]:
details = soup.find('div', id='detailBulletsWrapper_feature_div')

In [ ]:
publisher, year, isbn, edition, pages, language = '', '', '', '', '', ''
for li in details.div.ul.find_all('li'):
  field = str(li.span.span.string)
  content = str(li.span.find_all('span')[-1].string)
  if 'Editora' in field:  
    publication_info = content
    if ';' in publication_info:
      split_publication_info = publication_info.split(';')
      publisher = split_publication_info[0]
      year_and_edition_info = split_publication_info[1].split(' edição ')
      edition = year_and_edition_info[0]
      year = re.search(r'[0-9][0-9][0-9][0-9]', year_and_edition_info[1]).group(0)
    else:
      publisher = li.span.find_all('span')[-1].string
  if 'Idioma' in field:
    language = content
  if 'página' in content:
    pages = re.search(r'[0-9]+', content).group(0)
  if 'ISBN' in field:
    isbn = content
publisher, year, isbn, edition, pages, language

('Alta Books', '2022', '978-6555204865', ' 1ª', '256', 'Português')

#### Resultados

In [ ]:
def extract_amazon_info(soup):
  title = soup.find('span', id='productTitle').string.strip('\n ')
  info = ''
  description_div = soup.find('div', id='bookDescription_feature_div')
  if description_div:
    info = description_div.div.div.text.strip('\n ')
  authors_spans = soup.find('div', id='bylineInfo').find_all('span', class_='author')
  authors = []
  if authors_spans:
    for author in authors_spans:
      if 'Autor' in str(author.find('span', class_='contribution').span.string):
        if author.a:
          result = str(author.a.string)
          authors.append(result if 'Visite' not in result else result.replace('Visite a página de ', ''))
  price_span = soup.find('span', id='price')
  price = ''
  if price_span:
    price = str(price_span.string).replace(u'\xa0', u' ')
  details = soup.find('div', id='detailBulletsWrapper_feature_div')
  publisher, year, isbn, edition, pages, language = '', '', '', '', '', ''
  for li in details.div.ul.find_all('li'):
    field = str(li.span.span.string)
    content = str(li.span.find_all('span')[-1].string)
    if 'Editora' in field:  
      publication_info = content
      if ';' in publication_info:
        split_publication_info = publication_info.split(';')
        publisher = split_publication_info[0]
        if 'edição' in split_publication_info[1]:
          year_and_edition_info = split_publication_info[1].split(' edição ')
          edition = year_and_edition_info[0]
          year = re.search(r'[0-9][0-9][0-9][0-9]', year_and_edition_info[1]).group(0)
        else:
          year = re.search(r'[0-9][0-9][0-9][0-9]', split_publication_info[1]).group(0)
      else:
        publisher = li.span.find_all('span')[-1].string
    elif 'Idioma' in field:
      language = content
    elif 'página' in content:
      pages = re.search(r'[0-9]+', content).group(0)
    elif 'ISBN' in field:
      isbn = content
  return {
      "title": title,
      "publisher": publisher,
      "price": price,
      "info": info,
      "authors": authors,
      "year": year,
      "isbn": isbn,
      "edition": edition,
      "pages": pages,
      "language": language
  }

In [ ]:
file = f'{base_path}/amazon/2022-08-07_18.16.30.962588.html'
html = open(file,'r')
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
import os

In [ ]:
path = f'{base_path}/amazon'
for book in os.listdir(path):
  file = os.path.join(path, book)
  if '.html' in file:
    html = open(file,'r')
    print(extract_amazon_info(BeautifulSoup(html, 'html.parser')))

{'title': 'Disciplina É Liberdade: Manual do Campo', 'publisher': 'Alta Books', 'price': 'R$ 47,93', 'info': 'Os métodos de Jocko Willink para o sucesso nasceram nas equipes SEAL da marinha norte-americana, onde passou grande parte de sua vida adulta, alistando-se depois de sair do ensino médio e subindo na hierarquia até se tornar comandante das operações especiais com as mais altas condecorações na Guerra do Iraque. Em Disciplina É Liberdade, o coautor do livro Responsabilidade Extrema, best-seller do New York Times, descreve como vive segundo este mantra: as disciplinas mental e física que ele impõe a si mesmo para ter liberdade em todos os aspectos da vida. Muitos livros aconselham como superar obstáculos e alcançar seus objetivos, mas tal conselho costuma omitir o ingrediente mais crítico: a disciplina. Sem ela, não há progresso real. A obra Disciplina É Liberdade cobre tudo, inclusive estratégias e táticas para vencer a fraqueza, a procrastinação e o medo. Também apresenta um tre

### Cultura

#### Testes

In [ ]:
url = 'https://www3.livrariacultura.com.br/corpo-fala-61038/p'
opener = build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
soup = BeautifulSoup(opener.open(url), 'html.parser')

In [ ]:
title = soup.find("h1", {"class": "title_product"}).div.string

In [ ]:
title

'CORPO FALA'

In [ ]:
author_div = soup.find(class_='meta authors')
author = author_div.h2.a.string if author_div.h2.a else ''

In [ ]:
author

''

In [ ]:
publisher_div = soup.find('div', class_='publisher')
publisher = publisher_div.h2.div.a.string if publisher_div.h2.div else ''

In [ ]:
publisher

'VOZES'

In [ ]:
info_div = soup.find('div', id='info-product')
book_info = info_div.string if info_div else ''

In [ ]:
book_info

''

In [ ]:
prod_spec = soup.find(class_='section-produto-especificacoes')

In [ ]:
prod_spec

<section class="section-produto-especificacoes"><div class="container"><div class="info" id="produto-especificacoes"><div class="vtex-productSpecification" style="display: none;"><div id="caracteristicas"><h4 class="group Especificacoes">Especificações</h4><table cellspacing="0" class="group Especificacoes"><tr class="even"><th class="name-field Ano-de-Edicao">Ano de Edição</th><td class="value-field Ano-de-Edicao">2015</td></tr><tr><th class="name-field Colaborador">Colaborador</th><td class="value-field Colaborador">Autor:WEIL, PIERRE|Autor:TOMPAKOW, ROLAND</td></tr><tr class="even"><th class="name-field Editora">Editora</th><td class="value-field Editora">VOZES</td></tr><tr><th class="name-field ISBN">ISBN</th><td class="value-field ISBN">8532602088</td></tr><tr class="even"><th class="name-field Ano">Ano</th><td class="value-field Ano">2015</td></tr><tr><th class="name-field Edicao">Edição</th><td class="value-field Edicao">1</td></tr><tr class="even"><th class="name-field Origem">

In [ ]:
def treat_author(author_string: str):
  authors = author_string.split('|')
  result = []
  for a in authors:
    if not 'Autor' in a:
      continue
    name = a.replace('Autor:', '')
    name = name.split(', ')
    name.reverse()
    result.append(' '.join(name))
  return result

author_data = prod_spec.find('td', class_='value-field Colaborador')
authors = treat_author(author_data.string) if author_data else ''

In [ ]:
author_data

<td class="value-field Colaborador">Autor:WEIL, PIERRE|Autor:TOMPAKOW, ROLAND</td>

In [ ]:
authors

['PIERRE WEIL', 'ROLAND TOMPAKOW']

In [ ]:
isbn_data = prod_spec.find('td', class_='value-field ISBN')
isbn = isbn_data.string if isbn_data else ''

In [ ]:
isbn

'8532602088'

In [ ]:
year_data = prod_spec.find('td', class_='value-field Ano')
year = year_data.string if year_data else ''

In [ ]:
year

'2015'

In [ ]:
edition_data = prod_spec.find('td', class_='value-field Edicao')
edition = edition_data.string if edition_data else ''

In [ ]:
edition

'1'

In [ ]:
pages_data = prod_spec.find('td', class_='value-field Paginas')
pages = pages_data.string if pages_data else ''

In [ ]:
pages

'288'

In [ ]:
price_data = soup.find('em', class_='valor-por')
price_data

<em class="valor-por" productindex="0">Por: <strong class="skuBestPrice" productindex="0">R$ 45,50</strong></em>

In [ ]:
price = price_data.strong.string
price

'R$ 45,50'

#### Resultados
Juntando todas as informações numa função

In [ ]:
def extract_cultura_info(soup):
  def treat_author(author_string: str):
    authors = author_string.split('|')
    result = []
    for a in authors:
      if not 'Autor' in a:
        continue
      name = a.replace('Autor:', '')
      name = name.split(', ')
      name.reverse()
      result.append(' '.join(name))
    return result
  
  title = soup.find("h1", {"class": "title_product"}).div.string
  publisher_div = soup.find('div', class_='publisher')
  publisher = publisher_div.h2.div.a.string if publisher_div.h2.div else None
  info_div = soup.find('div', id='info-product')
  book_info = info_div.string if info_div else ''
  price_data = soup.find('em', class_='valor-por')
  price = price_data.strong.string if price_data else ''
  prod_spec = soup.find(class_='section-produto-especificacoes')
  if not publisher:
    publisher_data = prod_spec.find('td', class_='value-field Editora')
    publisher = publisher_data.string if publisher_data else ''
  author_data = prod_spec.find('td', class_='value-field Colaborador')
  authors = treat_author(author_data.string) if author_data else []
  isbn_data = prod_spec.find('td', class_='value-field ISBN')
  isbn = isbn_data.string if isbn_data else ''  
  language_data = prod_spec.find('td', class_='value-field Idioma')
  language = language_data.string if isbn_data else ''
  year_data = prod_spec.find('td', class_='value-field Ano')
  year = year_data.string if year_data else ''
  edition_data = prod_spec.find('td', class_='value-field Edicao')
  edition = edition_data.string if edition_data else ''
  pages_data = prod_spec.find('td', class_='value-field Paginas')
  pages = pages_data.string if pages_data else ''
  return {
      "title": title,
      "publisher": publisher,
      "price": price,
      "info": book_info,
      "authors": authors,
      "year": year,
      "isbn": isbn,
      "edition": edition,
      "pages": pages,
      "language": language
  }

In [ ]:
books = ['https://www3.livrariacultura.com.br/corpo-fala-61038/p',
  'https://www3.livrariacultura.com.br/naruto-gold-vol-1-2111902326/p',
  'https://www3.livrariacultura.com.br/o-ladrao-de-raios-capa-nova-42275209/p',
  'https://www3.livrariacultura.com.br/menos-que-um-2112320138/p',
  'https://www3.livrariacultura.com.br/thomas-sowell-a-biografia-2112320635/p',
  'https://www3.livrariacultura.com.br/gestao-com-gentileza-2112319991/p',
  'https://www3.livrariacultura.com.br/peaky-blinders-2112320052/p',
  'https://www3.livrariacultura.com.br/cacadas-de-pedrinho-2111919285/p',
  'https://www3.livrariacultura.com.br/dedoches-animais-2112297541/p?lista=1725',
  'https://www3.livrariacultura.com.br/stranger-things-cidade-nas-trevas-2112247884/p?lista=2099',
  'https://www3.livrariacultura.com.br/o-cavaleiro-preso-na-armadura-2112273695/p',
  'https://www3.livrariacultura.com.br/o-filho-do-terrorista-46016865/p',
  'https://www3.livrariacultura.com.br/a-verdade-e-outras-mentiras-46335302/p']

In [ ]:
len(books)

13

In [ ]:
for book in books:
  opener = build_opener()
  opener.addheaders = [('User-agent', 'Mozilla/5.0')]
  print(extract_cultura_info(BeautifulSoup(opener.open(book), 'html.parser')))

{'title': 'CORPO FALA', 'publisher': 'VOZES', 'price': 'R$ 45,50', 'info': '', 'authors': ['PIERRE WEIL', 'ROLAND TOMPAKOW'], 'year': '2015', 'isbn': '8532602088', 'edition': '1', 'pages': '288', 'language': 'PORTUGUÊS'}
{'title': 'NARUTO GOLD VOL. 1', 'publisher': 'PANINI LIVROS', 'price': 'R$ 23,73', 'info': '', 'authors': ['MASASHI KISHIMOTO'], 'year': '2022', 'isbn': '8542602412', 'edition': '1', 'pages': '192', 'language': 'PORTUGUÊS'}
{'title': 'O LADRÃO DE RAIOS - CAPA NOVA', 'publisher': 'INTRINSECA', 'price': 'R$ 38,43', 'info': '', 'authors': ['RICK RIORDAN'], 'year': '2014', 'isbn': '8580575397', 'edition': '1', 'pages': '400', 'language': 'PORTUGUÊS'}
{'title': 'MENOS QUE UM', 'publisher': 'LEYA.', 'price': 'R$ 44,98', 'info': '', 'authors': [], 'year': '', 'isbn': '', 'edition': '', 'pages': '', 'language': ''}
{'title': 'THOMAS SOWELL – A BIOGRAFIA', 'publisher': 'AVIS RARA', 'price': 'R$ 54,90', 'info': '', 'authors': [], 'year': '', 'isbn': '', 'edition': '', 'pages': '

### Companhia das letras

#### Testes

In [ ]:
url = 'https://www.companhiadasletras.com.br/livro/9788535923438/eu-sou-malala'
opener = build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
soup = BeautifulSoup(opener.open(url), 'html.parser')

In [ ]:
title = soup.find('div', class_='detalhe_livro_titulo').string
title

'EU SOU MALALA'

In [ ]:
authors_div = soup.find('div', class_='detalhe_livro_autor')
authors=[]
for a in authors_div.find_all('a'):
  authors.append(a.string)
authors

['Malala Yousafzai', 'Christina Lamb']

In [ ]:
price = soup.find('div', class_='preco').string
price

'R$ 52,90'

In [ ]:
details = soup.find_all('div', class_='bloco_txt_detalhe')
info = details[0].get_text()

In [ ]:
info

'\n          Quando o Talibã tomou controle do vale do Swat, uma menina levantou a voz. Malala Yousafzai recusou-se a permanecer em silêncio e lutou pelo seu direito à educação. Mas em 9 de outubro de 2012, uma terça-feira, ela quase pagou o preço com a vida. \nMalala foi atingida na cabeça por um tiro à queima-roupa dentro do ônibus no qual voltava da escola. Poucos acreditaram que ela sobreviveria.\nMas a recuperação milagrosa de Malala a levou em uma viagem extraordinária de um vale remoto no norte do Paquistão para as salas das Nações Unidas em Nova York. Aos dezesseis anos, ela se tornou um símbolo global de protesto pacífico e a candidata mais jovem da história a receber o Prêmio Nobel da Paz.\nEu sou Malala é a história de uma família exilada pelo terrorismo global, da luta pelo direito à educação feminina e dos obstáculos à valorização da mulher em uma sociedade que valoriza filhos homens. \nO livro acompanha a infância da garota no Paquistão, os primeiros anos de vida escolar,

In [ ]:
len(details)

4

In [ ]:
pages, year, isbn, publisher = '', '', '', ''
for span in details[1].find_all('span'):
  if span.string == 'Páginas:':
    pages = str(span.next_sibling).strip()
  if span.string == 'Lançamento:':
    year = str(span.next_sibling).split('/')[-1]
  if span.string == 'ISBN:':
    isbn = str(span.next_sibling).strip()
  if span.string == 'Selo:':
    publisher = str(span.next_sibling).strip()
pages, year, isbn, publisher

('360', '2013', '9788535923438', 'Companhia das Letras')

#### Resultados

In [ ]:
def extract_companhia_info(soup):
  title = soup.find('div', class_='detalhe_livro_titulo').string
  authors_div = soup.find('div', class_='detalhe_livro_autor')
  authors=[]
  if authors_div:
    for a in authors_div.find_all('a'):
      authors.append(a.string)
  price = soup.find('div', class_='preco').string
  details = soup.find_all('div', class_='bloco_txt_detalhe')
  book_info = details[0].get_text()
  pages, year, isbn, publisher = '', '', '', ''
  for span in details[1].find_all('span'):
    if span.string == 'Páginas:':
      pages = str(span.next_sibling).strip()
    if span.string == 'Lançamento:':
      year = str(span.next_sibling).split('/')[-1]
    if span.string == 'ISBN:':
      isbn = str(span.next_sibling).strip()
    if span.string == 'Selo:':
      publisher = str(span.next_sibling).strip()
  edition = ''
  return {
      "title": title,
      "authors": authors,
      "publisher": publisher,
      "price": price,
      "info": book_info,
      "year": year,
      "isbn": isbn,
      "edition": edition,
      "pages": pages,
      "language": ''
  }

In [ ]:
books = ['https://www.companhiadasletras.com.br/livro/9786559211852/saramago-150-os-seus-nomes',
  'https://www.companhiadasletras.com.br/livro/9786559790715/a-cultura-no-mundo-liquido-moderno-nova-edicao',
  'https://www.companhiadasletras.com.br/livro/9786559213474/metamaus',
  'https://www.companhiadasletras.com.br/livro/9786559790807/o-negocio-do-jair',
  'https://www.companhiadasletras.com.br/livro/9786559211845/todas-as-letras-nova-edicao-ampliada',
  'https://www.companhiadasletras.com.br/livro/9788535914849/1984',
  'https://www.companhiadasletras.com.br/livro/9788535928075/o-gene',
  'https://www.companhiadasletras.com.br/livro/9788582850541/impressoes-de-leitura-e-outros-textos-criticos',
  'https://www.companhiadasletras.com.br/livro/9788535912692/tintim-e-os-picaros',
  'https://www.companhiadasletras.com.br/livro/9788535921168/deus-essa-gostosa']

In [ ]:
for book in books:
  opener = build_opener()
  opener.addheaders = [('User-agent', 'Mozilla/5.0')]
  print(extract_companhia_info(BeautifulSoup(opener.open(book), 'html.parser')))

{'title': 'SARAMAGO – OS SEUS NOMES', 'authors': [], 'publisher': 'Companhia das Letras', 'price': 'R$ 199,90', 'info': '\n          José Saramago é, incontestavelmente, um dos grandes mestres da literatura. Único autor de língua portuguesa a receber o prêmio Nobel, ele deixou tesouros como Ensaio sobre a cegueira, O Evangelho segundo Jesus Cristo e O ano da morte de Ricardo Reis. Em 2022, celebra-se o centenário de nascimento desse homem que começou como serralheiro mecânico, tornou-se funcionário público, editor e, por fim, escritor – ofício que foi levado a outro patamar por suas mãos.Este álbum biográfico sem precedentes, dividido em quatro perspectivas – espaços/lugares, leituras/sentidos, escritas/criações e laços/pessoas –, traz a vida de José Saramago ao entrelaçar mais de duzentos nomes, entre cidades, marcos históricos, amigos, personagens. Lanzarote, Brasil, Blimunda, Deus, Judas, Fernando Pessoa, Marx, Jorge Amado, Pilar, todos estes relembrados por imagens do acervo da Fun

### Estante

#### Testes

In [ ]:
url = 'https://www.estantevirtual.com.br/leao-livros/adam-smith-a-riqueza-das-nacoes-v-1-2843205889?show_suggestion=0'
opener = build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
soup = BeautifulSoup(opener.open(url), 'html.parser')

In [ ]:
title = soup.find('h1', class_='livro-titulo').string.strip('\n ')

In [ ]:
title

'A Riqueza das Nacoes V. 1'

In [ ]:
authors = [soup.find('h2', class_='livro-autor').a.span.string]

In [ ]:
authors

['Adam Smith']

In [ ]:
price = soup.find('span', class_='livro-preco-valor').string.strip('\n ')
price

'R$ 19,90'

In [ ]:
specs = soup.find_all('p', class_='livro-specs')

In [ ]:
year, publisher, isbn, language, book_info = '', '', '', '', ''
for spec in specs:
  if 'Ano' in str(spec.span.string):
    year = spec.span.next_sibling.strip('\n ')
  elif 'Editora' in str(spec.span.string):
    publisher = spec.a.span.string
  elif 'ISBN' in str(spec.span.string):
    isbn = spec.span.next_sibling.strip('\n ')
  elif 'Idioma' in str(spec.span.string):
    language = spec.span.next_sibling.strip('\n ')
  elif 'Descrição' in str(spec.span.string):
    book_info = spec.find('span', class_='description-text').string.strip('\n ')
year, publisher, isbn, language, book_info

('1988',
 'Nova Cultural',
 '0000000000',
 'Português',
 'Capa mostrando rasuras  páginas amareladas pelo tempo sem riscos  ou rasuras muito bom no todo.')

#### Resultados

In [ ]:
def extract_estante_info(soup):
  title = soup.find('h1', class_='livro-titulo').string.strip('\n ')
  authors = soup.find('h2', class_='livro-autor').a.span.string.split('; ')
  price = soup.find('span', class_='livro-preco-valor').string.strip('\n ')
  specs = soup.find_all('p', class_='livro-specs')
  year, publisher, isbn, language, book_info = '', '', '', '', ''
  for spec in specs:
    if 'Ano' in str(spec.span.string):
      year = spec.span.next_sibling.strip('\n ')
    elif 'Editora' in str(spec.span.string):
      publisher = spec.a.span.string
    elif 'ISBN' in str(spec.span.string):
      isbn = spec.span.next_sibling.strip('\n ')
    elif 'Idioma' in str(spec.span.string):
      language = spec.span.next_sibling.strip('\n ')
    elif 'Descrição' in str(spec.span.string):
      book_info = spec.find('span', class_='description-text').string.strip('\n ')
  return {
      "title": title,
      "authors": authors,
      "publisher": publisher,
      "price": price,
      "info": book_info,
      "year": year,
      "isbn": isbn,
      "edition": '',
      "pages": '',
      "language": language
  }

In [ ]:
books = ['https://www.estantevirtual.com.br/livrariamosaico/jorge-amado-capitaes-da-areia-1%C2%AA-edicao-2466551162?show_suggestion=0',
  'https://www.estantevirtual.com.br/julianlivros/adam-smith-a-riqueza-das-nacoes-compendio-3271924492?show_suggestion=0',
  'https://www.estantevirtual.com.br/bibliomania/malala-yousafzai-eu-sou-malala-3260307694?show_suggestion=0',
  'https://www.estantevirtual.com.br/livraria-de-sueos/graciliano-ramos-vidas-secas-3272088254?show_suggestion=0',
  'https://www.estantevirtual.com.br/wtrlivros/graciliano-ramos-vidas-secas-edicao-comemorativa-80-anos-record-3275573386?show_suggestion=0',
  'https://www.estantevirtual.com.br/seboamigodolivro/laurentino-gomes-1889-3275008345?show_suggestion=0',
  'https://www.estantevirtual.com.br/wtrlivros/fernando-haddad-o-terceiro-excluido-contribuicao-para-uma-antropologia-dialetica-3276639950?show_suggestion=0',
  'https://www.estantevirtual.com.br/pook-book/laurentino-gomes-escravidao-volume-3-da-independencia-do-brasil-a-lei-aurea-3276914198?show_suggestion=0',
  'https://www.estantevirtual.com.br/discosafins/ivan-idoeta-francisco-capuano-elementos-de-eletronica-digital-3266565323?show_suggestion=0',
  'https://www.estantevirtual.com.br/cataventolivraria1/bram-stoker-dracula-3273679851?show_suggestion=0'
]

In [ ]:
for book in books:
  opener = build_opener()
  opener.addheaders = [('User-agent', 'Mozilla/5.0')]
  print(extract_estante_info(BeautifulSoup(opener.open(book), 'html.parser')))

{'title': 'Capitães da Areia - 1ª Edição', 'authors': ['Jorge Amado'], 'publisher': 'Livraria José Olympio', 'price': 'R$ 500,00', 'info': 'acervoap. Bom estado de conservação, sem páginas sublinhadas ou danificadas. Dedicatória ao antigo proprietário. Acabamento: Capa Dura de encadernação particular. Formato: Médio. Exemplar higienizado. Podemos enviar mais informações ou imagens através do whatsapp. 51998472886 - 201626881', 'year': '1937', 'isbn': '', 'edition': '', 'pages': '', 'language': 'Português'}
{'title': 'A Riqueza das Nações - Compêndio', 'authors': ['Adam Smith'], 'publisher': 'Não Encontrada', 'price': 'R$ 22,00', 'info': 'Código: 2294451 Nota: A presente obra encontra-se em bom estado de conservação, contém, apenas, algumas manchas amareladas causadas pelo tempo. \r\n Ano: 2014 Nº de páginas: 205 Formato: 14 x 21  Conservação da capa: Bom Estado Conservação do miolo: Bom Estado Acabamento: Brochura Idioma: Português Curiosidade: Editora: Hunter', 'year': 'n/d', 'isbn': 

### Saraiva

#### Testes

In [4]:
domain = 'saraiva'

In [5]:
file = f'{base_path}/{domain}/2022-08-09_10.15.44.558119.html'
html = open(file,'r')
soup = BeautifulSoup(html, 'html.parser')

In [8]:
title_element = soup.find('h1', class_='title')
title = title_element.string.strip() if title_element else ''
title

'A Desumanização'

In [9]:
price_p = soup.find('p', class_='price-destaque')
price = price_p.string.strip() if price_p else ''
price

'R$ 37,90'

In [11]:
info_div = soup.find('div', id='descricao')
info = info_div.text.strip('\n ') if info_div else ''
info

'Na paisagem gélida da Islândia, a menina Halla, de apenas onze anos de idade, busca compreender os sentimentos que surgem com o falecimento de sua irmã Sigridur. Vivendo a divisão permanente das “crianças espelhos”, Halla nos guia por impressões de transitoriedade e perda a partir do seu ponto de vista infantil e, por isso mesmo, cheio de uma simplicidade profundamente poética. O sofrimento do luto, a solidão e a violenta frieza da mãe se misturam com a paisagem inóspita da Terra do Gelo e, somados à narração lírica e melancólica de Valter Hugo Mãe, em que o desamparo dos personagens é superado por uma compreensão sublime e bela de sua condição, transformam esta obra em um primor da literatura contemporânea'

In [ ]:
features = soup.find('div', id='caracteristicas').div.div.div.table.tbody
features

In [36]:
year, authors, isbn, edition, language, pages, publisher = '', [], '', '', '', '', ''
if features:
  for tr in features.find_all('tr'):
    tds = tr.find_all('td')
    field, data = tds[0], tds[1]
    if not field or not data:
      continue
    field, data = field.string.lower(), data.string
    if 'ano da edição' in field:
      year = data.strip()
    elif 'isbn' in field:
      isbn = data.strip()
    elif 'idioma' in field:
      language = data.strip()
    elif 'número da edição' in field:
      edition = data.strip()
    elif 'páginas' in field:
      pages = data.strip()
    elif 'marca' in field:
      publisher = data.strip()
    elif 'autor' in field:
      name_list = data.split(',')
      names, first_names = name_list[::2], name_list[1::2]
      first_names.extend(['' for i in range(len(names)-len(first_names))])
      authors = [' '.join([y, x]).strip() for x, y in zip(names, first_names)]

year, authors, isbn, edition, language, pages, publisher

('2017',
 ['Valter Hugo Mãe'],
 '9788525063274',
 '1',
 'Português',
 '192',
 'Biblioteca Azul')